<a href="https://colab.research.google.com/github/wusinyee/MW-Portfolio/blob/main/TCCADBv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter-dash

In [ ]:
pip install dash

In [ ]:
!pip install streamlit plotly pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install dash plotly pandas numpy

In [17]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

# Load the data
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Define the color palette
colors = {
    'background': '#F0F8FF',  # Alice Blue
    'accent': '#1E90FF',      # Dodger Blue
    'text': '#333333',        # Dark Gray
    'highlight': '#87CEFA',   # Light Sky Blue
    'secondary': '#FF69B4'    # Hot Pink
}

def create_dashboard(df):
    # Create the dashboard layout
    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=(
            'Churn Distribution', 'Tenure vs Monthly Charges', 'Distribution of Tenure',
            'Churn by Contract Type', 'Internet Service Distribution', 'Correlation Heatmap',
            'Churn Rate by Gender', 'Churn Rate by Senior Citizen', 'Customer Demographics'
        ),
        specs=[
            [{'type': 'domain'}, {'type': 'xy'}, {'type': 'xy'}],
            [{'type': 'xy'}, {'type': 'domain'}, {'type': 'xy'}],
            [{'type': 'xy'}, {'type': 'xy'}, {'type': 'table'}]
        ],
        vertical_spacing=0.1,
        horizontal_spacing=0.05,
    )

    # 1. Churn Distribution
    churn_counts = df['Churn'].value_counts()
    fig.add_trace(go.Pie(
        labels=churn_counts.index,
        values=churn_counts.values,
        marker_colors=[colors['accent'], colors['secondary']],
        name='Churn Distribution',
        textinfo='label+percent',
        hole=0.3
    ), row=1, col=1)

    # 2. Tenure vs Monthly Charges
    fig.add_trace(go.Scatter(
        x=df['tenure'],
        y=df['MonthlyCharges'],
        mode='markers',
        marker=dict(
            color=df['Churn'].map({'Yes': colors['secondary'], 'No': colors['accent']}),
            size=5,
            opacity=0.6
        ),
        name='Tenure vs Monthly Charges'
    ), row=1, col=2)

    # 3. Distribution of Tenure
    fig.add_trace(go.Histogram(
        x=df['tenure'],
        marker_color=colors['accent'],
        name='Tenure Distribution',
        opacity=0.7
    ), row=1, col=3)

    # 4. Churn by Contract Type
    churn_by_contract = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack()
    fig.add_trace(go.Bar(
        x=churn_by_contract.index,
        y=churn_by_contract['Yes'],
        name='Churned',
        marker_color=colors['secondary'],
        opacity=0.7
    ), row=2, col=1)
    fig.add_trace(go.Bar(
        x=churn_by_contract.index,
        y=churn_by_contract['No'],
        name='Retained',
        marker_color=colors['accent'],
        opacity=0.7
    ), row=2, col=1)

    # 5. Internet Service Distribution
    internet_service = df['InternetService'].value_counts()
    fig.add_trace(go.Pie(
        labels=internet_service.index,
        values=internet_service.values,
        marker_colors=[colors['accent'], colors['secondary'], colors['highlight']],
        name='Internet Service',
        textinfo='label+percent',
        hole=0.3
    ), row=2, col=2)

    # 6. Correlation Heatmap
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    correlation = numeric_df.corr()
    fig.add_trace(go.Heatmap(
        z=correlation.values,
        x=correlation.columns,
        y=correlation.columns,
        colorscale='Blues',
        name='Correlation Heatmap'
    ), row=2, col=3)

    # 7. Churn Rate by Gender
    churn_by_gender = df.groupby('gender')['Churn'].value_counts(normalize=True).unstack()
    fig.add_trace(go.Bar(
        x=churn_by_gender.index,
        y=churn_by_gender['Yes'],
        name='Churn Rate by Gender',
        marker_color=colors['accent'],
        opacity=0.7
    ), row=3, col=1)

    # 8. Churn Rate by Senior Citizen
    churn_by_senior = df.groupby('SeniorCitizen')['Churn'].value_counts(normalize=True).unstack()
    fig.add_trace(go.Bar(
        x=['Non-Senior', 'Senior'],
        y=churn_by_senior['Yes'],
        name='Churn Rate by Senior Citizen',
        marker_color=colors['accent'],
        opacity=0.7
    ), row=3, col=2)

    # 9. Customer Demographics table
    demographics = df[['gender', 'SeniorCitizen', 'Partner', 'Dependents']].describe().T
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Metric'] + list(demographics.columns),
                fill_color=colors['highlight'],
                align='left',
                font=dict(color=colors['text'])
            ),
            cells=dict(
                values=[demographics.index] + [demographics[col] for col in demographics.columns],
                fill_color=colors['background'],
                align='left',
                font=dict(color=colors['text'])
            )
        ),
        row=3, col=3
    )

    # Update layout
    fig.update_layout(
        height=1200,
        width=1600,
        showlegend=False,
        title=dict(
            text="Telco Customer Churn Dashboard",
            x=0.5,
            y=0.98,
            xanchor='center',
            yanchor='top',
            font=dict(size=24, color=colors['text'])
        ),
        font=dict(family="Arial, sans-serif", size=12, color=colors['text']),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        barmode='stack',
    )

    # Update axes
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor=colors['highlight'], title_font=dict(size=12))
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor=colors['highlight'], title_font=dict(size=12))

    # Update subplot titles
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color=colors['text'])

    # Add hover templates
    fig.update_traces(
        hovertemplate="<b>%{y:.2f}</b>",
        selector=dict(type='bar')
    )
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Monthly Charges: $%{y:.2f}",
        selector=dict(type='scatter')
    )

    return fig

# Create and show the dashboard
dashboard = create_dashboard(df)
dashboard.show()